### **Setting Up the Development Environment**

#### Loading Required Data

In [8]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
pd.set_option('display.max_colwidth', None)

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

import langchain
langchain.verbose = True


In [9]:
load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [10]:
file_name = r"C:\github_repos\mars_question_answering_app_rags\data\MARS_Benchmark_Diciembre (1).csv"
try:
    df = pd.read_csv(file_name, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_name, encoding='latin1')

df.shape

(5006, 54)

In [11]:
df.head()

,Id,Date,Time,Media Type,Site Name,Site Domain,Mention URL,Publisher Name,Publisher Username,title,...,Youtube Comments,Youtube Favorites,User Age,User Gender,User Family Status,User Marital Status,User Affinities,User Jobs,User biography tags,Media URL
0,2.670000e+11,31/12/2023,22:58:00 -0600 CST,Social Media,Twitter,http://www.twitter.com,http://twitter.com/jfeyg7/status/1741685322897920364#1649664281242136576_1741665455285166431,jacky,jfeyg7,"La @ARSL0V3R RT @yutaomega: me quiero follar al osito bimbo, sus deliciosas curvas me excitan demasiado, tanto que me hacen llegar al orgasmo de solo verlas, me gustaría darle unas fuertes nalgadas a sus blancas y suaves pompas joder me tienes demente ni puedo verte en el supermercado porque se me pone dura",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.670000e+11,31/12/2023,22:53:00 -0600 CST,Social Media,Twitter,http://www.twitter.com,http://twitter.com/ferconsciente/status/1741683980246610073#1367253737912336396_1741610337135186198,Ferconsciente,ferconsciente,"RT @contralinea: LO MÁS LEÍDO | En México, el Poder Judicial de la Federación insiste en beneficiar a Bayer-Monsanto con permitir la introducción masiva del carcinógeno #glifosato al país #Contralínea #PeriodismodeInvestigación https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVgC537o",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.670000e+11,31/12/2023,22:53:00 -0600 CST,Social Media,Twitter,http://www.twitter.com,http://twitter.com/joeg51287877/status/1741683934164107384#1272068416594014208_1741610337135186198,joeg,joeg51287877,"RT @contralinea: LO MÁS LEÍDO | En México, el Poder Judicial de la Federación insiste en beneficiar a Bayer-Monsanto con permitir la introducción masiva del carcinógeno #glifosato al país #Contralínea #PeriodismodeInvestigación https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVgC537o",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.670000e+11,31/12/2023,22:49:00 -0600 CST,Social Media,Twitter,http://www.twitter.com,http://twitter.com/DeivSol0/status/1741683033193820525#1537965259863887873_1741610337135186198,David ?4T,DeivSol0,"RT @contralinea: LO MÁS LEÍDO | En México, el Poder Judicial de la Federación insiste en beneficiar a Bayer-Monsanto con permitir la introducción masiva del carcinógeno #glifosato al país #Contralínea #PeriodismodeInvestigación https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVgC537o",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.670000e+11,31/12/2023,22:49:00 -0600 CST,Social Media,Twitter,http://www.twitter.com,http://twitter.com/ALFREDOGARCIASA/status/1741682952734482498#92381621_1741610337135186198,ALFREDO GARCIA S. #RedAMLOVE,ALFREDOGARCIASA,"RT @contralinea: LO MÁS LEÍDO | En México, el Poder Judicial de la Federación insiste en beneficiar a Bayer-Monsanto con permitir la introducción masiva del carcinógeno #glifosato al país #Contralínea #PeriodismodeInvestigación https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVgC537o",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Load a CSV file into a list of Documents

Each document represents one row of the CSV file. Every row is converted into a key/value pair and outputted to a new line in the document’s page_content.

In [15]:
loader = CSVLoader(file_path=file_name, encoding='latin1')

try:
    docs = loader.load()
except UnicodeDecodeError:

    loader = CSVLoader(file_path=file_name, encoding='ISO-8859-1')
    docs = loader.load()

In [16]:
len(docs)

5006

In [17]:
print(docs[0].page_content[:500])

ï»¿Id: 2.67E+11
Date: 31/12/2023
Time: 22:58:00 -0600 CST
Media Type: Social Media
Site Name: Twitter
Site Domain: http://www.twitter.com
Mention URL: http://twitter.com/jfeyg7/status/1741685322897920364#1649664281242136576_1741665455285166431
Publisher Name: jacky
Publisher Username: jfeyg7
title: La @ARSL0V3R RT @yutaomega: me quiero follar al osito bimbo, sus deliciosas curvas me excitan demasiado, tanto que me hacen llegar al orgasmo de solo verlas, me gustarÃ­a darle unas fuertes nalgadas a


### Split Documents

In [19]:
chunk_size = 512 # chunk_size can be modified between 512, 558 until 1236 which belongs to OpenAIEmbeddings (however I recommend 512)
#chunk_size = 558
chunk_overlap = 32

c_text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len
)

r_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
    add_start_index = True,
)

In [20]:
pages = c_text_splitter.split_documents(docs)

print(pages[0])
print(pages[1])

Created a chunk of size 568, which is longer than the specified 512
Created a chunk of size 578, which is longer than the specified 512
Created a chunk of size 566, which is longer than the specified 512
Created a chunk of size 576, which is longer than the specified 512
Created a chunk of size 536, which is longer than the specified 512
Created a chunk of size 557, which is longer than the specified 512
Created a chunk of size 567, which is longer than the specified 512
Created a chunk of size 529, which is longer than the specified 512
Created a chunk of size 539, which is longer than the specified 512
Created a chunk of size 522, which is longer than the specified 512
Created a chunk of size 532, which is longer than the specified 512
Created a chunk of size 533, which is longer than the specified 512
Created a chunk of size 543, which is longer than the specified 512
Created a chunk of size 774, which is longer than the specified 512
Created a chunk of size 639, which is longer tha

page_content='ï»¿Id: 2.67E+11\nDate: 31/12/2023\nTime: 22:58:00 -0600 CST\nMedia Type: Social Media\nSite Name: Twitter\nSite Domain: http://www.twitter.com\nMention URL: http://twitter.com/jfeyg7/status/1741685322897920364#1649664281242136576_1741665455285166431\nPublisher Name: jacky\nPublisher Username: jfeyg7' metadata={'source': 'C:\\github_repos\\mars_question_answering_app_rags\\data\\MARS_Benchmark_Diciembre (1).csv', 'row': 0}
page_content='Publisher Username: jfeyg7\ntitle: La @ARSL0V3R RT @yutaomega: me quiero follar al osito bimbo, sus deliciosas curvas me excitan demasiado, tanto que me hacen llegar al orgasmo de solo verlas, me gustarÃ\xada darle unas fuertes nalgadas a sus blancas y suaves pompas joder me tienes demente ni puedo verte en el supermercado porque se me pone dura' metadata={'source': 'C:\\github_repos\\mars_question_answering_app_rags\\data\\MARS_Benchmark_Diciembre (1).csv', 'row': 0}


In [21]:
pages = r_text_splitter.split_documents(docs)

print(pages[0])
print(pages[1])

page_content='ï»¿Id: 2.67E+11\nDate: 31/12/2023\nTime: 22:58:00 -0600 CST\nMedia Type: Social Media\nSite Name: Twitter\nSite Domain: http://www.twitter.com\nMention URL: http://twitter.com/jfeyg7/status/1741685322897920364#1649664281242136576_1741665455285166431\nPublisher Name: jacky\nPublisher Username: jfeyg7' metadata={'source': 'C:\\github_repos\\mars_question_answering_app_rags\\data\\MARS_Benchmark_Diciembre (1).csv', 'row': 0, 'start_index': 0}
page_content='Publisher Username: jfeyg7\ntitle: La @ARSL0V3R RT @yutaomega: me quiero follar al osito bimbo, sus deliciosas curvas me excitan demasiado, tanto que me hacen llegar al orgasmo de solo verlas, me gustarÃ\xada darle unas fuertes nalgadas a sus blancas y suaves pompas joder me tienes demente ni puedo verte en el supermercado porque se me pone dura' metadata={'source': 'C:\\github_repos\\mars_question_answering_app_rags\\data\\MARS_Benchmark_Diciembre (1).csv', 'row': 0, 'start_index': 266}


In [22]:
len(docs)

5006

In [23]:
len(pages)

21091

## Vectorstore and Embeddings

In [24]:
embedding = OpenAIEmbeddings()

c:\anaconda3\envs\mars\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [25]:
persist_directory = 'persist_chroma'

In [26]:
vectordb = Chroma.from_documents(
    documents=pages,
    embedding=embedding,
    persist_directory=persist_directory
)

In [27]:
print(vectordb._collection.count())

21091


In [28]:
len(pages)

21091

In [29]:
vectordb.persist()

As good as the Prompt is, so will be the result.
Prompt Techniques Also Must Be Considered:
- Direct Instruction.
- Open-Ended Questions.
- Multiple-Choice Questions.
- Leading Questions.
- Hypothetical Scenarios.
- Sequential or Step-by-Step Prompts.
- Reflective Questions.
- Comparative Analysis.
- Analogies or Metaphors.
- Requesting Examples or Illustrations.
- Elaboration Requests.

In [30]:
# Example 1
question = "Explicame que se dice sobre Bayer-Monsanto?"

In [31]:
docs = vectordb.similarity_search(question,k=3)
docs

[Document(page_content='Mention Content: @SCJN SIEMPRE A FAVOR DE EMPRESAS EXTRANJERAS https://x.com/contralinea/status/1741610337135186198?s=20 RT @contralinea: LO MÃ\x81S LEÃ\x8dDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introducciÃ³n masiva del carcinÃ³geno #glifosato al paÃ\xads #ContralÃ\xadnea #PeriodismodeInvestigaciÃ³n', metadata={'row': 293, 'source': 'C:\\github_repos\\mars_question_answering_app_rags\\data\\MARS_Benchmark_Diciembre (1).csv', 'start_index': 854}),
 Document(page_content='title: RT @contralinea: LO MÃ\x81S LEÃ\x8dDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introducciÃ³n masiva del carcinÃ³geno #glifosato al paÃ\xads #ContralÃ\xadnea #PeriodismodeInvestigaciÃ³n https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVg

### **Retrieval with MMR (Maximal Marginal Relevance)**

#### Semantic Search (SS) VS Maximal Marginal Relevance (MMR)

#### Load a Persist VectorDB (Chroma - AI-Native Open-Source Embedding Database)

In [32]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [33]:
docs_ss = vectordb.similarity_search(question,k=3)
docs_ss

[Document(page_content='Mention Content: @SCJN SIEMPRE A FAVOR DE EMPRESAS EXTRANJERAS https://x.com/contralinea/status/1741610337135186198?s=20 RT @contralinea: LO MÃ\x81S LEÃ\x8dDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introducciÃ³n masiva del carcinÃ³geno #glifosato al paÃ\xads #ContralÃ\xadnea #PeriodismodeInvestigaciÃ³n', metadata={'row': 293, 'source': 'C:\\github_repos\\mars_question_answering_app_rags\\data\\MARS_Benchmark_Diciembre (1).csv', 'start_index': 854}),
 Document(page_content='title: RT @contralinea: LO MÃ\x81S LEÃ\x8dDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introducciÃ³n masiva del carcinÃ³geno #glifosato al paÃ\xads #ContralÃ\xadnea #PeriodismodeInvestigaciÃ³n https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVg

In [34]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
docs_mmr

[Document(page_content='Mention Content: @SCJN SIEMPRE A FAVOR DE EMPRESAS EXTRANJERAS https://x.com/contralinea/status/1741610337135186198?s=20 RT @contralinea: LO MÃ\x81S LEÃ\x8dDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introducciÃ³n masiva del carcinÃ³geno #glifosato al paÃ\xads #ContralÃ\xadnea #PeriodismodeInvestigaciÃ³n', metadata={'row': 293, 'source': 'C:\\github_repos\\mars_question_answering_app_rags\\data\\MARS_Benchmark_Diciembre (1).csv', 'start_index': 854}),
 Document(page_content='title: RT @contralinea: LO MÃ\x81S LEÃ\x8dDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introducciÃ³n masiva del carcinÃ³geno #glifosato al paÃ\xads #ContralÃ\xadnea #PeriodismodeInvestigaciÃ³n https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVg

### **RAGs For Question Answering (Q&A)**

### Using LangChain Default Prompt

In [35]:
llm_name = "gpt-3.5-turbo" # The Model Can Be Changed As Required

llm = ChatOpenAI(model_name=llm_name, temperature=1)

c:\anaconda3\envs\mars\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [36]:
qa_chain_default = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k":3}), # The 'k' Value Can Be Increased to Improve The RetrievalQA
    chain_type="stuff",
    return_source_documents=True
)

In [37]:
# Example 2
question = "Explicame de forma precisa, resumida y muy concreta que se dice sobre Bayer-Monsanto?"

result = qa_chain_default({"query": question})
result

c:\anaconda3\envs\mars\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Mention Content: RT @contralinea: LO MÃS LEÃDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introducciÃ³n masiva del carcinÃ³geno #glifosato al paÃ­s #ContralÃ­nea #PeriodismodeInvestigaciÃ³n https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVgC537o
Topics: MARS _Benchmark
Subtopics: Bayer
Classifiers: 
Classifiers tags:

Mention Content: RT @contralinea: LO MÃS LEÃDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introdu

{'query': 'Explicame de forma precisa, resumida y muy concreta que se dice sobre Bayer-Monsanto?',
 'result': 'Según el artículo mencionado, el Poder Judicial de la Federación en México está insistiendo en permitir la introducción masiva del carcinógeno glifosato al país, lo cual beneficiaría a Bayer-Monsanto. Sin embargo, no se proporcionan más detalles sobre las acciones específicas de Bayer-Monsanto ni sobre las implicaciones de la introducción de glifosato en México.',
 'source_documents': [Document(page_content='Mention Content: RT @contralinea: LO MÃ\x81S LEÃ\x8dDO | En MÃ©xico, el Poder Judicial de la FederaciÃ³n insiste en beneficiar a Bayer-Monsanto con permitir la introducciÃ³n masiva del carcinÃ³geno #glifosato al paÃ\xads #ContralÃ\xadnea #PeriodismodeInvestigaciÃ³n https://contralinea.com.mx/interno/semana/tribunal-insiste-en-aprobar-el-glifosato-y-tirar-decreto-presidencial/?fbclid=IwAR0vW67EZE5-g6TkA94_k0zg2d43U7KlpX_XxI3eJy6E-iUZylvZVgC537o\nTopics: MARS _Benchmark\nSub

In [38]:
print(result.get("result"))

Según el artículo mencionado, el Poder Judicial de la Federación en México está insistiendo en permitir la introducción masiva del carcinógeno glifosato al país, lo cual beneficiaría a Bayer-Monsanto. Sin embargo, no se proporcionan más detalles sobre las acciones específicas de Bayer-Monsanto ni sobre las implicaciones de la introducción de glifosato en México.


In [39]:
def pretty_print(text, words_per_line=15):
    words = text.split()


    for i in range(0, len(words), words_per_line):

        line = ' '.join(words[i:i+words_per_line])
        print(line)

In [40]:
pretty_print(result.get("result"))

Según el artículo mencionado, el Poder Judicial de la Federación en México está insistiendo en
permitir la introducción masiva del carcinógeno glifosato al país, lo cual beneficiaría a Bayer-Monsanto. Sin
embargo, no se proporcionan más detalles sobre las acciones específicas de Bayer-Monsanto ni sobre las
implicaciones de la introducción de glifosato en México.


### Other Examples

In [ ]:
# Example 3
question = "Genera 5 bullets más importantes sobre MARS y la sustentabilidad, por favor?"

result = qa_chain_default({"query": question})
result

In [ ]:
print(result.get("result"))

In [ ]:
pretty_print(result.get("result"))

In [ ]:
# Example 4
question = "Por favor explicame aspectos negativos que se aborden en todo el documento sobre Nestlé, despliega la salida en formato de bullets."

result = qa_chain_default({"query": question})
result

In [ ]:
print(result.get("result"))

In [ ]:
pretty_print(result.get("result"))

In [ ]:
# Example 5
question = "Por favor explicame aspectos negativos que se aborden en todo el documento sobre Nestlé, despliega la salida en formato de bullets. Para esta respuesta considera la columna Sentiment del documento original"

result = qa_chain_default({"query": question})
result

In [ ]:
print(result.get("result"))

In [ ]:
pretty_print(result.get("result"))